In [18]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
from googletrans import Translator, LANGUAGES
from gtts import gTTS
import pyperclip
import os
import time
from playsound import playsound
import threading

class TranslationApp:
    def __init__(self, root):
        self.root = root
        self.translator = Translator()
        self.history = []
        self.current_audio_file = None

        self.root.title("L.E.X.A. (Linguistic Expert Assistant)")
        self.root.geometry("1000x700")
        self.root.configure(bg="#1a1a2e")
        self.root.resizable(True, True)
        
        self.create_widgets()
        self.setup_menu()
        
    def create_widgets(self):
        style = ttk.Style()
        style.theme_use('alt')
        
        style.configure('.', background="#1a1a2e", foreground="#e6e6e6")
        style.configure('TFrame', background="#1a1a2e")
        style.configure('TLabel', background="#1a1a2e", foreground="#e6e6e6", 
                       font=('Segoe UI', 10))
        style.configure('TButton', font=('Segoe UI', 10, 'bold'), padding=8, 
                       background="#4cc9f0", foreground="#1a1a2e")
        style.map('TButton', background=[('active', '#4895ef')])
        
        # Custom style for comboboxes
        style.configure('TCombobox', fieldbackground="#16213e", foreground="#e6e6e6", 
                       font=('Segoe UI', 10), padding=5)
        style.map('TCombobox', fieldbackground=[('readonly', '#16213e')],
                 selectbackground=[('readonly', '#4a4e69')],
                 selectforeground=[('readonly', '#ffffff')])
        
        style.configure('Vertical.TScrollbar', background="#4a4e69", troughcolor="#1a1a2e")
        
        main_frame = ttk.Frame(self.root, padding=(20, 15))
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        header_frame = ttk.Frame(main_frame)
        header_frame.pack(fill=tk.X, pady=(0, 20))
        
        ttk.Label(header_frame, text="L.E.X.A. (Linguistic Expert Assistant)", 
                 font=('Segoe UI', 18, 'bold'), foreground="#4cc9f0").pack(side=tk.LEFT)
        
        input_frame = ttk.LabelFrame(main_frame, text=" Source Text ", padding=15, 
                                    style='Card.TFrame')
        input_frame.pack(fill=tk.X, pady=(0, 20))
        
        self.source_text = scrolledtext.ScrolledText(input_frame, height=8, font=('Segoe UI', 11),
                                                    wrap=tk.WORD, padx=10, pady=10,
                                                    bg="#16213e", fg="#e6e6e6", 
                                                    insertbackground="#4cc9f0",
                                                    selectbackground="#4a4e69")
        self.source_text.pack(fill=tk.BOTH, expand=True)
        
        lang_frame = ttk.Frame(main_frame)
        lang_frame.pack(fill=tk.X, pady=(0, 20))
        
        # Source language combobox with custom styling
        self.src_lang = ttk.Combobox(lang_frame, values=['Auto Detect'] + sorted(LANGUAGES.values()), 
                                     state='readonly', width=28, font=('Segoe UI', 10))
        self.src_lang.current(0)
        self.src_lang.configure(style='Custom.TCombobox')
        self.src_lang.pack(side=tk.LEFT, padx=5)
        
        # Create custom style for Auto Detect combobox
        style.configure('AutoDetect.TCombobox', fieldbackground="#4cc9f0", foreground="#1a1a2e",
                       font=('Segoe UI', 10, 'bold'), padding=5)
        style.map('AutoDetect.TCombobox', fieldbackground=[('readonly', '#4cc9f0')],
                 selectbackground=[('readonly', '#4895ef')],
                 selectforeground=[('readonly', '#1a1a2e')])
        
        ttk.Label(lang_frame, text="→", font=('Segoe UI', 14), foreground="#4cc9f0").pack(side=tk.LEFT, padx=10)
        
        # Destination language combobox
        self.dest_lang = ttk.Combobox(lang_frame, values=sorted(LANGUAGES.values()), 
                                      state='readonly', width=28, font=('Segoe UI', 10))
        self.dest_lang.current(sorted(LANGUAGES.values()).index('english'))
        self.dest_lang.pack(side=tk.LEFT, padx=5)
        
        output_frame = ttk.LabelFrame(main_frame, text=" Translated Text ", padding=15)
        output_frame.pack(fill=tk.BOTH, expand=True)
        
        self.translated_text = scrolledtext.ScrolledText(output_frame, height=8, font=('Segoe UI', 11),
                                                      wrap=tk.WORD, padx=10, pady=10,
                                                      bg="#16213e", fg="#e6e6e6", 
                                                      state=tk.DISABLED)
        self.translated_text.pack(fill=tk.BOTH, expand=True)
        
        btn_frame = ttk.Frame(main_frame)
        btn_frame.pack(fill=tk.X, pady=(20, 5))
        
        buttons = [
            ("Translate", self.translate),
            ("Speak Translation", self.speak_translation),
            ("Copy to Clipboard", self.copy_to_clipboard),
            ("Clear All", self.clear_all)
        ]
        
        for text, command in buttons:
            btn = ttk.Button(btn_frame, text=text, command=command, style='Accent.TButton')
            btn.pack(side=tk.LEFT, padx=5, ipadx=10)
        
        self.status_bar = ttk.Frame(self.root, height=25, style='Status.TFrame')
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        self.status_label = ttk.Label(self.status_bar, text="Ready", font=('Segoe UI', 9),
                                     foreground="#a8a8a8", style='Status.TLabel')
        self.status_label.pack(side=tk.LEFT, padx=10)
        
        style.configure('Card.TFrame', background="#16213e", borderwidth=0, 
                        relief=tk.SOLID, bordercolor="#4a4e69")
        style.configure('Status.TFrame', background="#0f3460")
        style.configure('Status.TLabel', background="#0f3460")
        style.configure('Accent.TButton', background="#4cc9f0", foreground="#1a1a2e")
        
        # Update the combobox style when selection changes
        self.src_lang.bind("<<ComboboxSelected>>", self.update_combobox_style)
        
    def update_combobox_style(self, event):
        if self.src_lang.get() == 'Auto Detect':
            self.src_lang.configure(style='AutoDetect.TCombobox')
        else:
            self.src_lang.configure(style='TCombobox')

    def setup_menu(self):
        menu_bar = tk.Menu(self.root, tearoff=0, bg="#16213e", fg="#e6e6e6", 
                          activebackground="#4cc9f0", activeforeground="#1a1a2e")
        
        file_menu = tk.Menu(menu_bar, tearoff=0, bg="#16213e", fg="#e6e6e6",
                           activebackground="#4cc9f0", activeforeground="#1a1a2e")
        file_menu.add_command(label="Export History", command=self.export_history)
        file_menu.add_separator()
        file_menu.add_command(label="Exit", command=self.root.quit)
        menu_bar.add_cascade(label="File", menu=file_menu)
        
        help_menu = tk.Menu(menu_bar, tearoff=0, bg="#16213e", fg="#e6e6e6",
                           activebackground="#4cc9f0", activeforeground="#1a1a2e")
        help_menu.add_command(label="About", command=self.show_about)
        menu_bar.add_cascade(label="Help", menu=help_menu)
        
        self.root.config(menu=menu_bar)
    
    def translate(self):
        text = self.source_text.get("1.0", tk.END).strip()
        if not text:
            self.show_error("Please enter text to translate!")
            return
            
        try:
            src_lang = 'auto' if self.src_lang.get() == 'Auto Detect' else \
                      list(LANGUAGES.keys())[list(LANGUAGES.values()).index(self.src_lang.get().lower())]
            dest_lang = list(LANGUAGES.keys())[list(LANGUAGES.values()).index(self.dest_lang.get().lower())]
            
            result = self.translator.translate(text, src=src_lang, dest=dest_lang)
            
            self.history.append({
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S"),
                'source': f"{LANGUAGES[result.src].title()} ({result.src})",
                'target': f"{LANGUAGES[result.dest].title()} ({result.dest})",
                'original': text,
                'translated': result.text,
                'pronunciation': result.pronunciation
            })
            
            self.translated_text.config(state=tk.NORMAL)
            self.translated_text.delete("1.0", tk.END)
            self.translated_text.insert(tk.END, result.text)
            self.translated_text.config(state=tk.DISABLED)
            
            if self.src_lang.get() == 'Auto Detect':
                self.status_label.config(text=f"Detected language: {LANGUAGES[result.src].title()}")
            else:
                self.status_label.config(text="Translation complete!")
                
        except Exception as e:
            self.show_error(f"Translation error: {str(e)}")
            
    def speak_translation(self):
        if not self.translated_text.get("1.0", tk.END).strip():
            self.show_error("No translation to speak!")
            return
            
        try:
            text = self.translated_text.get("1.0", tk.END)
            dest_lang = list(LANGUAGES.keys())[list(LANGUAGES.values()).index(self.dest_lang.get().lower())]
            
            filename = f"tts_{int(time.time())}.mp3"
            
            def tts_thread():
                tts = gTTS(text=text, lang=dest_lang, slow=False)
                tts.save(filename)
                playsound(filename)
                os.remove(filename)
                
            threading.Thread(target=tts_thread, daemon=True).start()
            self.status_label.config(text="Playing audio...")
            
        except Exception as e:
            self.show_error(f"Text-to-speech error: {str(e)}")
            
    def copy_to_clipboard(self):
        text = self.translated_text.get("1.0", tk.END).strip()
        if text:
            pyperclip.copy(text)
            self.status_label.config(text="Copied to clipboard!")
            
    def clear_all(self):
        self.source_text.delete("1.0", tk.END)
        self.translated_text.config(state=tk.NORMAL)
        self.translated_text.delete("1.0", tk.END)
        self.translated_text.config(state=tk.DISABLED)
        self.status_label.config(text="Ready")
        
    def export_history(self):
        try:
            with open("translation_history.txt", "w", encoding="utf-8") as f:
                for entry in self.history:
                    f.write(f"[{entry['timestamp']}] {entry['source']} → {entry['target']}\n")
                    f.write(f"Original: {entry['original']}\n")
                    f.write(f"Translated: {entry['translated']}\n")
                    if entry['pronunciation']:
                        f.write(f"Pronunciation: {entry['pronunciation']}\n")
                    f.write("\n")
            self.status_label.config(text="History exported to translation_history.txt")
        except Exception as e:
            self.show_error(f"Export error: {str(e)}")
            
    def show_error(self, message):
        messagebox.showerror("Error", message)
        self.status_label.config(text="Error occurred - see message box")
        
    def show_about(self):
        about_text = """L.E.X.A. (Linguistic Expert Assistant)
        
Features:
- Text translation between 100+ languages
- Text-to-speech capability
- Translation history
- Simple and intuitive interface
        
Created with Python and Tkinter"""
        messagebox.showinfo("About L.E.X.A.", about_text)

if __name__ == "__main__":
    root = tk.Tk()
    app = TranslationApp(root)
    root.mainloop()